# Colab for roop-unleashed - Gradio version
https://github.com/C0untFloyd/roop-unleashed


Installing & preparing requirements

# Installation 

In [ ]:
from IPython.display import clear_output
import torch
import codecs

if torch.cuda.is_available():
    !apt-get update
    !apt-get install -y nvidia-cuda-toolkit
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

git_repo_rot13 = 'uggcf://tvguho.pbz/P0hagSyblq/ebbc-hayrnfurq.tvg'
git_repo = codecs.decode(git_repo_rot13, 'rot 13')

directory_rot13 = 'ebbc-hayrnfurq'
directory = codecs.decode(directory_rot13, 'rot 13')

# Clone the repository
!git clone $git_repo /kaggle/working/$directory

# Change directory to the cloned repository
%cd /kaggle/working/$directory

# Install requirements if they exist
!pip install -r requirements.txt

# Install additional package
!pip install onnxruntime-gpu


# Launch with Pinggy

In [ ]:
from multiprocessing import Process
import sys
import time
import codecs

# Ensure the log file exists and is empty
!touch log.txt
open('log.txt', 'w').close()

def run_app():
    directory_rot13 = 'ebbc-hayrnfurq'
    directory = codecs.decode(directory_rot13, 'rot 13')
    cmd = f"python /kaggle/working/$directory/run.py & ssh -o StrictHostKeyChecking=no -p 80 -R 0:localhost:7860 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    while not found:
        with open('log.txt', 'r') as file:
            end_word = '.pinggy.io'
            for line in file:
                start_index = line.find("http://")
                if start_index != -1:
                    end_index = line.find(end_word, start_index)
                    if end_index != -1:
                        print("😁 😁 😁")
                        print("URL: " + line[start_index:end_index + len(end_word)])
                        print("😁 😁 😁")
                        found = True
                        break
        if not found:
            time.sleep(2)  # Sleep before checking again

p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()



# Run With Local Tunnel

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request
from colorama import Fore, Style

# Install localtunnel
subprocess.run(["npm", "install", "-g", "localtunnel"])

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    # Fetch and display the external IP address
    external_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
    print(Fore.GREEN + "\nIP: ", Fore.RED, external_ip, "\n", Style.RESET_ALL)

    # Start the LocalTunnel process
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')

# Start the iframe thread to check for the local server
threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

# Start your main application (e.g., Gradio app)
subprocess.run(["python", "run.py"])

# Flask For Ngrok

In [ ]:
!pip install flask pyngrok

from flask import Flask
from pyngrok import ngrok
import threading

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

def run_flask():
    app.run(port=7860)

# Start ngrok and expose the Flask app
def run_ngrok():
    public_url = ngrok.connect(port=7860)
    print(" * Running on ngrok:", public_url)

# Start the Flask server and ngrok in separate threads
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

ngrok_thread = threading.Thread(target=run_ngrok)
ngrok_thread.start()

# Run with Ngrok

In [ ]:
!pip install pyngrok

from pyngrok import ngrok
import os
import threading
import subprocess
import codecs

# Function to establish ngrok tunnel
def start_ngrok_tunnel():
    global ngrok_tunnel
    try:
        ngrok_tunnel = ngrok.connect(addr="7861", proto="http")
        print("Ngrok Tunnel URL:", ngrok_tunnel.public_url)
    except Exception as e:
        print("Error starting ngrok tunnel:", e)

# Set your ngrok authtoken here
ngrok.set_auth_token("2cEUTBoN0RMOyEw0bM15YBLuikc_6BMTZJoVvQEjDFgunUqbz")

# Terminate any existing ngrok sessions
try:
    active_tunnels = ngrok.get_tunnels()
    for tunnel in active_tunnels:
        ngrok.disconnect(tunnel.public_url)
except Exception as e:
    print("Error retrieving or disconnecting active ngrok tunnels:", e)

# Start ngrok tunnel in a separate thread and wait for it to establish
ngrok_tunnel = None
ngrok_thread = threading.Thread(target=start_ngrok_tunnel)
ngrok_thread.start()
ngrok_thread.join()

# Decode directory using ROT13
def decode_directory(rot13_encoded_directory):
    return codecs.decode(rot13_encoded_directory, 'rot_13')

# Directory with ROT13 encoding
directory_rot13 = 'ebbc-hayrnfurq'
directory = decode_directory(directory_rot13)
print(f"Decoded directory: {directory}")

# Change directory to the decoded directory
os.chdir(f"/kaggle/working/{directory}")

# Execute the command to start the run.py script with CUDA execution providers
command = "python run.py"
subprocess.run(command, shell=True)

# Disconnect the ngrok tunnel
if ngrok_tunnel:
    try:
        ngrok.disconnect(ngrok_tunnel.public_url)
        print("Ngrok tunnel disconnected.")
    except Exception as e:
        print("Error disconnecting ngrok tunnel:", e)
